In [1]:
import numpy as np
import tensorflow as tf
from matplotlib import pyplot as plt

print(tf.__version__)

1.14.0


# Scaled dot product attention

In [2]:
def scaled_dot_product_attention(q, k, v, mask):
    """Calculate the attention weights.
    q, k, v must have matching leading dimensions.
    k, v must have matching penultimate dimension, i.e.: seq_len_k = seq_len_v.
    The mask has different shapes depending on its type(padding or look ahead) 
    but it must be broadcastable for addition.

    Args:
    q: query shape == (..., seq_len_q, depth)
    k: key shape == (..., seq_len_k, depth)
    v: value shape == (..., seq_len_v, depth_v)
    mask: Float tensor with shape broadcastable 
          to (..., seq_len_q, seq_len_k). Defaults to None.

    Returns:
    output, attention_weights
    """

    matmul_qk = tf.matmul(q, k, transpose_b=True)  # (..., seq_len_q, seq_len_k)

    # scale matmul_qk
    dk = tf.cast(tf.shape(k)[-1], tf.float32)
    scaled_attention_logits = matmul_qk / tf.math.sqrt(dk)

    # add the mask to the scaled tensor.
    if mask is not None:
        scaled_attention_logits += (mask * -1e9)  

    # softmax is normalized on the last axis (seq_len_k) so that the scores
    # add up to 1.
    attention_weights = tf.nn.softmax(scaled_attention_logits, axis=-1)  # (..., seq_len_q, seq_len_k)

    output = tf.matmul(attention_weights, v)  # (..., seq_len_q, depth_v)

    return output, attention_weights

In [ ]:
def print_out(q, k, v):
    temp_out, temp_attn = scaled_dot_product_attention(
      q, k, v, None)
    print ('Attention weights are:')
    print (temp_attn)
    print ('Output is:')
    print (temp_out)

np.set_printoptions(suppress=True)

temp_k = tf.constant([[10,0,0],
                      [0,10,0],
                      [0,0,10],
                      [0,0,10]], dtype=tf.float32)  # (4, 3)

temp_v = tf.constant([[   1,0],
                      [  10,0],
                      [ 100,5],
                      [1000,6]], dtype=tf.float32)  # (4, 2)

# This `query` aligns with the second `key`,
# so the second `value` is returned.
temp_q = tf.constant([[0, 10, 0]], dtype=tf.float32)  # (1, 3)
print_out(temp_q, temp_k, temp_v)

# Multi-head attention

In [3]:
class MultiHeadAttention(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads):
        super(MultiHeadAttention, self).__init__()
        self.num_heads = num_heads
        self.d_model = d_model

        assert d_model % self.num_heads == 0

        self.depth = d_model // self.num_heads

        self.wq = tf.keras.layers.Dense(d_model)
        self.wk = tf.keras.layers.Dense(d_model)
        self.wv = tf.keras.layers.Dense(d_model)

        self.dense = tf.keras.layers.Dense(d_model)

    def split_heads(self, x, batch_size):
        """Split the last dimension into (num_heads, depth).
        Transpose the result such that the shape is (batch_size, num_heads, seq_len, depth)
        """
        x = tf.reshape(x, (batch_size, -1, self.num_heads, self.depth))
        return tf.transpose(x, perm=[0, 2, 1, 3])

    def call(self, v, k, q, mask):
        batch_size = tf.shape(q)[0]

        q = self.wq(q)  # (batch_size*top_n, seq_len, d_model)
        k = self.wk(k)  # (batch_size*top_n, seq_len, d_model)
        v = self.wv(v)  # (batch_size*top_n, seq_len, d_model)

        q = self.split_heads(q, batch_size)  # (batch_size, num_heads, seq_len_q, depth)
        k = self.split_heads(k, batch_size)  # (batch_size, num_heads, seq_len_k, depth)
        v = self.split_heads(v, batch_size)  # (batch_size, num_heads, seq_len_v, depth)

        # scaled_attention.shape == (batch_size, num_heads, seq_len_q, depth)
        # attention_weights.shape == (batch_size, num_heads, seq_len_q, seq_len_k)
        scaled_attention, attention_weights = scaled_dot_product_attention(
            q, k, v, mask)

        scaled_attention = tf.transpose(scaled_attention, perm=[0, 2, 1, 3])  # (batch_size, seq_len_q, num_heads, depth)

        concat_attention = tf.reshape(scaled_attention, 
                                      (batch_size, -1, self.d_model))  # (batch_size, seq_len_q, d_model)

        output = self.dense(concat_attention)  # (batch_size, seq_len_q, d_model)

        return output, attention_weights

In [ ]:
temp_mha = MultiHeadAttention(d_model=512, num_heads=8)
y = tf.random.uniform((1, 60, 512))  # (batch_size, encoder_sequence, d_model)
out, attn = temp_mha(y, k=y, q=y, mask=None)
out.shape, attn.shape

# Point wise feed forward network

In [4]:
def point_wise_feed_forward_network(d_model, dff):
    return tf.keras.Sequential([
      tf.keras.layers.Dense(dff, activation='relu'),  # (batch_size, seq_len, dff)
      tf.keras.layers.Dense(d_model)  # (batch_size, seq_len, d_model)
    ])

# Encoder layer

In [5]:
class EncoderLayer(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads, dff, rate=0.1):
        super(EncoderLayer, self).__init__()

        self.mha = MultiHeadAttention(d_model, num_heads)
        self.ffn = point_wise_feed_forward_network(d_model, dff)

        self.layernorm1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)

        self.dropout1 = tf.keras.layers.Dropout(rate)
        self.dropout2 = tf.keras.layers.Dropout(rate)

    def call(self, x, training, mask):

        attn_output, _ = self.mha(x, x, x, mask)  # (batch_size, input_seq_len, d_model)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(x + attn_output)  # (batch_size, input_seq_len, d_model)

        ffn_output = self.ffn(out1)  # (batch_size, input_seq_len, d_model)
        ffn_output = self.dropout2(ffn_output, training=training)
        out2 = self.layernorm2(out1 + ffn_output)  # (batch_size, input_seq_len, d_model)

        return out2

# Encoder_base

In [ ]:
class Encoder_base(tf.keras.layers.Layer):
    def __init__(self, num_layers, d_model, num_heads, dff, input_vocab_size, 
               rate=0.1):
        super(Encoder_base, self).__init__()

        self.d_model = d_model
        self.num_layers = num_layers

        self.embedding = tf.keras.layers.Embedding(input_vocab_size, d_model)
        self.pos_encoding = positional_encoding(input_vocab_size, self.d_model)


        self.enc_layers = [EncoderLayer(d_model, num_heads, dff, rate) 
                           for _ in range(num_layers)]

        self.dropout = tf.keras.layers.Dropout(rate)

    def call(self, x, training, mask):

        seq_len = tf.shape(x)[1]

        # adding embedding and position encoding.
        x = self.embedding(x)  # (batch_size, input_seq_len, d_model)
        x *= tf.math.sqrt(tf.cast(self.d_model, tf.float32))
        x += self.pos_encoding[:, :seq_len, :]

        x = self.dropout(x, training=training)

        for i in range(self.num_layers):
            x = self.enc_layers[i](x, training, mask)

        return x  # (batch_size, input_seq_len, d_model)

In [ ]:
sample_encoder = Encoder_base(num_layers=2, d_model=512, num_heads=8, 
                         dff=2048, input_vocab_size=8500)

sample_encoder_output = sample_encoder(tf.random.uniform((64, 62)), 
                                       training=False, mask=None)

print (sample_encoder_output.shape)  # (batch_size, input_seq_len, d_model)

# Encoder_N

In [6]:
class Encoder(tf.keras.layers.Layer):
    def __init__(self, num_layers, d_model, num_heads, dff, input_vocab_size, 
               rate=0.1):
        super(Encoder, self).__init__()

        self.d_model = d_model
        self.num_layers = num_layers

        self.enc_layers = [EncoderLayer(d_model, num_heads, dff, rate) 
                           for _ in range(num_layers)]

        self.dropout = tf.keras.layers.Dropout(rate)

    def call(self, x, training, mask):

        x = self.dropout(x, training=True)

        for i in range(self.num_layers):
            x = self.enc_layers[i](x, True, False)

        return x  # (batch_size, input_seq_len, d_model)

In [ ]:
# 未使用
class Encoder(tf.keras.layers.Layer):
    def __init__(self, num_layers, d_model, num_heads, dff, num_labels,
               rate=0.1):
        super(Encoder, self).__init__()

        self.d_model = d_model
        self.num_layers = num_layers
        self.num_labels = num_labels

        self.enc_layers = [EncoderLayer(d_model, num_heads, dff, rate) 
                           for _ in range(num_layers)]

        self.dropout = tf.keras.layers.Dropout(rate)
        self.out = tf.keras.layers.Dense(self.num_labels)

    def call(self, x): #, training, mask):
        
        batch = tf.shape(x)[0]
        topn = tf.shape(x)[1]
        seq_len = tf.shape(x)[2]
        
        x = tf.reshape(x, (batch, topn, -1))
        
        # adding embedding and position encoding.
        # x = self.embedding(x)  # (batch_size, input_seq_len, d_model)
        # x *= tf.math.sqrt(tf.cast(self.d_model, tf.float32))
        # x += self.pos_encoding[:, :seq_len, :]
        
        # x = self.dropout(x, training=training)
        x = self.dropout(x, training=True)

        for i in range(self.num_layers):
            # x = self.enc_layers[i](x, training, mask)
            x = self.enc_layers[i](x, True, False)
        
        x = tf.reshape(x, (batch, topn, seq_len, -1))
        
        # 添加 self-attention
        
        # 添加softmax
        x = tf.reshape(x, (batch, -1))
        x = self.out(x)
        
        return x  # (batch_size, labels) // (batch_size, input_seq_len, d_model)

In [ ]:
sample_encoder = Encoder(num_layers=2, d_model=500, num_heads=5, 
                         dff=2048, input_vocab_size=8500)

sample_encoder_output = sample_encoder(tf.random.uniform((64, 20, 10*50)), 
                                       training=False, mask=None)

print(sample_encoder_output.shape)  # (batch_size, input_seq_len, d_model)

# Metrics
**Macro-f1**:  
$\text{Macro}-F_{1}=\frac{1}{|\mathcal{S}|} \sum_{t \in \mathcal{S}} \frac{2 P_{t} R_{t}}{P_{t}+R_{t}} \\
P_{t}=\frac{T P_{t}}{T P_{t}+F P_{t}} \\
R_{t}=\frac{T P_{t}}{T P_{t}+F N_{t}}$  


**Micro-f1**:  
$Micro-F_{1}=\frac{2 P R}{P+R} \\ 
\text{Precision(P)}=\frac{\sum_{t \in \mathcal{S}} T P_{t}}{\sum_{t \in \mathcal{S}} T P_{t}+F P_{t}} \\
\text{Recall}(R)=\frac{\sum_{t \in \mathcal{S}} T P_{t}}{\sum_{t \in \mathcal{S}} T P_{t}+F N_{t}}
$

``` ptyhon
y_true = np.array([[1,0,1,1,0],[1,1,0,1,1]])
y_pred = np.array([[0,1,1,1,0],[1,1,1,0,1]])

macro_f1 0.7333333333333333
micro_f1 0.7142857142857143
```

In [406]:
from tensorflow.keras.callbacks import Callback
from sklearn.metrics import f1_score, confusion_matrix

class Metrics(Callback):
    
    # model.fit can't support auto set valdidation_data
    # handle set val_data in __init__
    def __init__(self, val_data):
        super().__init__()
        self.validation_data = val_data
        
    def f1(self, y_true, y_pred):
        assert np.shape(y_true) == np.shape(y_pred)
        
        tp = np.sum(y_true * y_pred, axis = 0)
        tpfp = np.sum(y_true, axis=0)
        tpfn = np.sum(y_pred, axis=0)

        """Macro_F1 metric.
        
        for each label, compute a binary classification f1, and then,
        average all of them.
        """
        p = tp / tpfp 
        r = tp / tpfn
        macro_f1 = np.mean(2*p*r/(p+r))

        """Micro_F1 metric.
        
        Statitic all TPs, FPs, FNs, and then use f1 fomulation.
        
        """
        p = np.sum(tp)/np.sum(tpfp)
        r = np.sum(tp)/np.sum(tpfn)
        micro_f1 = 2*p*r/(p+r)

        return macro_f1, micro_f1

    def on_train_begin(self,logs={}):
        self.val_micro_f1s = []
        self.val_macro_f1s = []
        #self.val_recalls = []
        #self.val_precisions = []

    def on_epoch_end(self, epoch, logs={}):
        val_predict = (np.asarray(self.model.predict(
            self.validation_data[0]))).round()
        val_targ = self.validation_data[1]
        
        _val_macro_f1,_val_micro_f1 = self.f1(val_targ, val_predict)
        self.val_macro_f1s.append(_val_macro_f1)
        self.val_micro_f1s.append(_val_micro_f1)
        #self.val_recalls.append(_val_recall)
        #self.val_precisions.append(_val_precision)
        return

# Set hyperparameters

In [7]:
# d_model = seq_lens*embedding
topn = 100
seq_lens = 20
embedding = 50

num_layers = 1
d_model = seq_lens * embedding
dff = 512
num_heads = 10
num_labels = 126
dff = 1024
# input_vocab_size = tokenizer_pt.vocab_size + 2
# target_vocab_size = tokenizer_en.vocab_size + 2
dropout_rate = 0.1

# Model

In [407]:
# tf.reset_default_graph()
tff = tf.keras.Sequential()
tff.add(tf.keras.layers.Reshape((topn,d_model)))
tff.add(Encoder(num_layers, d_model, num_heads, dff, dropout_rate))
tff.add(tf.keras.layers.Flatten())
tff.add(tf.keras.layers.Dense(num_labels))

tff.compile(loss=tf.losses.sigmoid_cross_entropy,
              optimizer='adam')

# x = np.random.uniform(size=(64,topn,seq_lens,embedding)) # batch,topn,sequence_length,embdding_dim
# y = np.random.randint(num_labels,size=(64))

# dont use auto split train valid dataset in .fit
val_metric = Metrics([x[-2:],y[-2:]])
his = tff.fit(x[:4],y[:4],
          batch_size=2,
          epochs=2,
          callbacks=[val_metric])

tff.summary()

Epoch 1/2
4/4 [==============================] - 11s 3s/sample - loss: 1.3314


/home/nocater/anaconda3/envs/ml/lib/python3.6/site-packages/ipykernel_launcher.py:24: RuntimeWarning: invalid value encountered in true_divide
/home/nocater/anaconda3/envs/ml/lib/python3.6/site-packages/ipykernel_launcher.py:25: RuntimeWarning: divide by zero encountered in true_divide
/home/nocater/anaconda3/envs/ml/lib/python3.6/site-packages/ipykernel_launcher.py:25: RuntimeWarning: invalid value encountered in true_divide
/home/nocater/anaconda3/envs/ml/lib/python3.6/site-packages/ipykernel_launcher.py:26: RuntimeWarning: invalid value encountered in true_divide


Epoch 2/2
4/4 [==============================] - 8s 2s/sample - loss: 0.7625
Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
reshape_3 (Reshape)          multiple                  0         
_________________________________________________________________
encoder_3 (Encoder)          multiple                  6058024   
_________________________________________________________________
flatten_3 (Flatten)          multiple                  0         
_________________________________________________________________
dense_27 (Dense)             multiple                  12600126  
Total params: 18,658,150
Trainable params: 18,658,150
Non-trainable params: 0
_________________________________________________________________


# load data

In [11]:
import os
PATH = r'/mnt/d/Dataset/rcv1/'

# for file in os.listdir(PATH+'matrix/'):
x = np.load(PATH+'/matrix/range2000_4000.npy')
y = np.load(PATH+'/label/label_range2000_4000.npy')
x.shape,y.shape

((1551, 100, 20, 50), (1551, 126))